In [1]:
from nfc.ndef.m2m import m2m_certificate_format as m2m
from nfc.ndef.m2m import x509_to_m2m_conversion

In [2]:
import base64
import binascii
from pyasn1.type import univ
from pyasn1.codec.der import encoder as der_encoder
from pyasn1.codec.der import decoder as der_decoder

In [3]:
m2m_cert = m2m.m2m_certificate_from_file("m2m_certificate.pem")

Below, note that the decoded result is only the Sequence of TBSCertificate, rather than the [Application 20] IMPLICIT SEQUENCE that is the full Certificate. 

This means the rest of the secuance is omitted, e.g. the signature of the now Signed certificate. 

In [4]:
print(m2m_cert.prettyPrint())
pubkey = bytes(m2m_cert[7]) # This is a hardcoded value, only known because pubkey happens to be the 7th field due to some optionals not being ued
pubkey

Sequence:
 <no-name>=0
 <no-name>=0x00000000000000000000000000000000075bcd15
 <no-name>=1.2.840.10045.4.3.2
 <no-name>=0x06082a8648ce3d030107
 <no-name>=Sequence:
  <no-name>=b'US'
  <no-name>=b'ACME corp.'
  <no-name>=b'Fairfield'

 <no-name>=Sequence:
  <no-name>=b'US'
  <no-name>=b'ACME corp.'
  <no-name>=b'Fairfield'

 <no-name>=1.2.840.10045.4.3.2
 <no-name>=0x3059301306072a8648ce3d020106082a8648ce3d03010703420004c828d5ab30ea0a7e4a4b6418983e9b09a8d8d4bf3e95aff9d282520f0e672999bdcea794231c29976d791b3f9a78124e107102d3a7c78e76f74cd75072ed5a2a
 <no-name>=0x01
 <no-name>=0xa0
 <no-name>=2.5.29.32.0
 <no-name>=2.16.840.1.114513.29.37



b'0Y0\x13\x06\x07*\x86H\xce=\x02\x01\x06\x08*\x86H\xce=\x03\x01\x07\x03B\x00\x04\xc8(\xd5\xab0\xea\n~JKd\x18\x98>\x9b\t\xa8\xd8\xd4\xbf>\x95\xaf\xf9\xd2\x82R\x0f\x0eg)\x99\xbd\xce\xa7\x94#\x1c)\x97my\x1b?\x9ax\x12N\x10q\x02\xd3\xa7\xc7\x8ev\xf7L\xd7Pr\xedZ*'

In [5]:
signature = b'0E\x02!\x00\xf9%y\xe3p\xf9\xadC$\x97\xc8\xec\xee\xff\xab\x90\x0e\xd6\t\xf6J\xaa+\xc5B\xe2c\x01\xc9,*\xd1\x02 B\x87\x1d6\xb9\x0f\xccN\x9b\xa2~\xd4n\xdc\xc5A\xc0\x9b<c\xba\xaf\x1b\xdc\x1b\tE\x82\xc9K\x89J'

In [6]:
bytes_to_sign = b'\x9c\x15l\x01ultimaker.nl:material1\x00\x000451BD6A154A81A\xd5\xe6\xcc!\xa1\xe5\xcd\xfe\x15\xed\x8a3\xc3OW\xa2\xa7\xb4\xb7\x8a8\xc3\xcb\x00\x00TESTBATCH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

So, now we must check that the signature matches with the public key and the bytes_to_sign

In [7]:
m2m.verify_signature(bytes_to_sign, signature, public_key_path='public.pem')

True

In [8]:
def signature_from_m2m_pem(pem_path):
    cert_bytes = m2m.m2m_bytes_from_file(pem_path)
    decoded = der_decoder.decode(cert_bytes)
    certificate = decoded[0]
    
    reencoded = der_encoder.encode(certificate)
    # Because we only get back thr TBScertificate, 
    # see how long that is (when encoded) and extract the rest of the original bytes. 
    # That encodes the signature. 
    rest = cert_bytes[len(reencoded)+1:] 
#     signature_der = der_decoder.decode(rest)[0]
    return rest

signature = signature_from_m2m_pem("m2m_certificate.pem")
print(binascii.hexlify(signature))
print()

decoded_sig = der_decoder.decode(signature)[0]
print("cACalcValue = "+decoded_sig.prettyPrint())
print()
signature_int_seq = der_decoder.decode(decoded_sig.asOctets())[0]
print("Signature integers: "+signature_int_seq.prettyPrint())

b'04483046022100a106708f42cebf29f6080c0535818d7a6461150b23474e45819f495bcaa77804022100a161b744c1ca01f7d938af7e41143488652bdbd63c352c426228dd8b00ea9e9e'

cACalcValue = 0x3046022100a106708f42cebf29f6080c0535818d7a6461150b23474e45819f495bcaa77804022100a161b744c1ca01f7d938af7e41143488652bdbd63c352c426228dd8b00ea9e9e

Signature integers: Sequence:
 <no-name>=72833746562193434703155487150859005291729736000969498120385690645925387008004
 <no-name>=72995017660455601282376060995272935200071354704479255565849720793553758232222



In [9]:
m2m_cert = m2m.m2m_certificate_from_file("m2m_certificate.pem")
tbs_bytes = der_encoder.encode(m2m_cert)
print(m2m_cert.prettyPrint())
print(binascii.hexlify(tbs_bytes))
sig = decoded_sig.asOctets()
print(binascii.hexlify(sig))
 
# Was this cert signed by the CA that has the privkey of this pubkey?
m2m.verify_signature(tbs_bytes, sig, "public.pem")

Sequence:
 <no-name>=0
 <no-name>=0x00000000000000000000000000000000075bcd15
 <no-name>=1.2.840.10045.4.3.2
 <no-name>=0x06082a8648ce3d030107
 <no-name>=Sequence:
  <no-name>=b'US'
  <no-name>=b'ACME corp.'
  <no-name>=b'Fairfield'

 <no-name>=Sequence:
  <no-name>=b'US'
  <no-name>=b'ACME corp.'
  <no-name>=b'Fairfield'

 <no-name>=1.2.840.10045.4.3.2
 <no-name>=0x3059301306072a8648ce3d020106082a8648ce3d03010703420004c828d5ab30ea0a7e4a4b6418983e9b09a8d8d4bf3e95aff9d282520f0e672999bdcea794231c29976d791b3f9a78124e107102d3a7c78e76f74cd75072ed5a2a
 <no-name>=0x01
 <no-name>=0xa0
 <no-name>=2.5.29.32.0
 <no-name>=2.16.840.1.114513.29.37

b'7481ea3081e7020100041400000000000000000000000000000000075bcd1506082a8648ce3d040302040a06082a8648ce3d030107301b130255530c0a41434d4520636f72702e0c09466169726669656c64301b130255530c0a41434d4520636f72702e0c09466169726669656c6406082a8648ce3d040302045b3059301306072a8648ce3d020106082a8648ce3d03010703420004c828d5ab30ea0a7e4a4b6418983e9b09a8d8d4bf3e95aff9d282520f

False

Next is checking the certificate what is next in the chain, used to sign this M2M certificate. However, I don't have a certificate chain yet, just a single self-signed sertificate. 

In [10]:
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, PrivateFormat
from cryptography.hazmat.primitives.serialization import NoEncryption, KeySerializationEncryption
from cryptography.exceptions import InvalidSignature
import datetime
import uuid

one_day = datetime.timedelta(1, 0, 0)

In [11]:
def certificate_to_pem(certificate, pem_path):
    with open(pem_path, 'wb') as pem_file:
        pem_file.write(certificate.public_bytes(Encoding.PEM))

In [12]:
root_private_key = ec.generate_private_key(ec.SECP256R1, default_backend())
root_public_key = root_private_key.public_key()

builder = x509.CertificateBuilder()
builder = builder.subject_name(x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, u'Root')]))
builder = builder.issuer_name(x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, u'Root')]))
builder = builder.not_valid_before(datetime.datetime.today() - one_day)
builder = builder.not_valid_after(datetime.datetime(2018, 8, 2))
builder = builder.serial_number(int(uuid.uuid4()))
builder = builder.public_key(root_public_key)
builder = builder.add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True)

root_certificate = builder.sign(
    private_key=root_private_key, 
    algorithm=hashes.SHA256(),
    backend=default_backend())

certificate_to_pem(root_certificate, "root.pem")

In [13]:
middle_private_key = ec.generate_private_key(ec.SECP256R1, default_backend())
middle_public_key = middle_private_key.public_key()

builder = x509.CertificateBuilder()
builder = builder.subject_name(x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, u'Middle')]))
builder = builder.issuer_name(x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, u'Root')]))
builder = builder.not_valid_before(datetime.datetime.today() - one_day)
builder = builder.not_valid_after(datetime.datetime(2018, 8, 2))
builder = builder.serial_number(int(uuid.uuid4()))
builder = builder.public_key(middle_public_key)
builder = builder.add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True)

middle_certificate = builder.sign(
    private_key=root_private_key, #The middle cert is signed by the root!
    algorithm=hashes.SHA256(),
    backend=default_backend())

certificate_to_pem(root_certificate, "middle.pem")

middle_privkey_path = "/tmp/middle_privkey.key"
with open(middle_privkey_path, 'wb') as middle_privkey_file:
    middle_privkey_file.write(middle_private_key.private_bytes(encoding=Encoding.PEM, 
                                                      encryption_algorithm=NoEncryption(),
                                                      format=PrivateFormat.PKCS8))

In [14]:
leaf_private_key = ec.generate_private_key(ec.SECP256R1, default_backend())
leaf_public_key = leaf_private_key.public_key()

builder = x509.CertificateBuilder()
builder = builder.subject_name(x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, u'Leaf')]))
builder = builder.issuer_name(x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, u'Middle')]))
builder = builder.not_valid_before(datetime.datetime.today() - one_day)
builder = builder.not_valid_after(datetime.datetime(2018, 8, 2))
builder = builder.serial_number(int(uuid.uuid4()))
builder = builder.public_key(leaf_public_key)
builder = builder.add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True)

leaf_certificate = builder.sign(
    private_key=middle_private_key, #The leaf cert is signed by the middle 
    algorithm=hashes.SHA256(),
    backend=default_backend())

certificate_to_pem(root_certificate, "leaf.pem")

pubkey_path = "/tmp/pubkey.key"
with open(pubkey_path, 'wb') as pubkey_file:
    pubkey_file.write(leaf_public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo))
    
privkey_path = "/tmp/privkey.key"
with open(privkey_path, 'wb') as privkey_file:
    privkey_file.write(leaf_private_key.private_bytes(encoding=Encoding.PEM, 
                                                      encryption_algorithm=NoEncryption(),
                                                      format=PrivateFormat.PKCS8))

x509_to_m2m_conversion.M2mConverter.x509_pem_to_m2m_pem("leaf.pem", "leaf.m2m.pem", middle_privkey_path)

In [15]:
# The root certificate is not to be included according to the M2M spec, but we do here for ease
chain = {leaf_certificate:middle_certificate, 
         middle_certificate:root_certificate, 
         root_certificate:root_certificate} # Each certificate links to the certificate it was signed with

In [16]:
signature = m2m.generate_signature(bytes_to_sign, private_key_path=privkey_path)

In [17]:
signature

b'0E\x02!\x00\xae\x80\xd6\xdcf\xbc\xe5\xf3\xef\x1d<\xe4\x07C\x14\xf5C\xf2\x8f%\x88\\5\xe3\xcb|\x89\xb9\x8f\x12^*\x02 )\x81W\xc7>\x19\xe7\xc1\x05/\x1f:\x07\x1a\xf9e\xc7\xbf\xac\xf6\t\xafs#iG\x83H\xee$\x802'

In [18]:
signature_OK = m2m.verify_signature(bytes_to_sign, signature, public_key_path=pubkey_path)
signature_OK

True

In [19]:
verified = middle_certificate.public_key().verify(
    signature=leaf_certificate.signature,
    data=leaf_certificate.tbs_certificate_bytes,
    signature_algorithm=ec.ECDSA(algorithm=hashes.SHA256()))

In [20]:
# $ openssl verify -CAfile root.pem -untrusted middle.pem leaf.pem
def verify_certificate_by_signer(certificate, signer):
    # TODO: check that no certificate is on a Certificate Revocation List (CRL)
    try:
        signer.public_key().verify(signature=certificate.signature,
                                 data=certificate.tbs_certificate_bytes,
                                 signature_algorithm=ec.ECDSA(algorithm=hashes.SHA256()))
    except InvalidSignature as bad_sig:
        print(certificate, signer)
        raise bad_sig

# Link each issuer name to its certificate
issuers = {"Leaf":leaf_certificate,
           "Middle":middle_certificate,
           "Root":root_certificate}
    
def verify_chain(certificate):
    issuer_name = certificate.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value
    issuer_certificate = issuers[issuer_name]
    verify_certificate_by_signer(certificate, issuer_certificate)
    
    if not issuer_certificate == certificate:
        verify_chain(issuer_certificate)
    
verify_chain(leaf_certificate)
print("OK")

OK
